In [1]:
from IPython.display import HTML

HTML('''<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');
      $('#toggleButton').val('Show Code')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Hide Code')
    }
    code_shown = !code_shown
  }

  $( document ).ready(function(){
    code_shown=false;
    $('div.input').hide()
  });
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Code"></form>''')

In [2]:
from IPython.display import display
from sympy import *
import sympy.physics.units as unit
init_printing()
import math
import numpy as np

from sympy.physics.units import convert_to


In [3]:
# data collections
sub_dict = {}
d = {}
eq_set = []
var_set = []
# utility functions
def add_equation(equation, solve_for):
    dict_entry = {}
    dict_entry['eq']=equation
    dict_entry['var']=solve_for
    dict_entry['result'] = None
    d[dict_entry['var']]=dict_entry
    
def save_results(results_list):
    i=0
    for entry in d.values():
        entry['result'] = results_list[i]
        i+=1

## Parameters

Payload mass

In [4]:
m_payload = symbols('M_{payload}')
m_payload_eq = Eq(m_payload,18*unit.kg)
add_equation(m_payload_eq,m_payload)
# sub_dict[m_payload]=solve(m_payload_calc)[0]
m_payload_eq

M_{payload} = 18⋅kilogram

Fly time

In [5]:
fly_time = symbols('T_{fly}')
fly_time_var = 2
fly_time_eq = Eq(fly_time,fly_time_var*unit.hour)
add_equation(fly_time_eq,fly_time)
# sub_dict[m_payload]=solve(m_payload_calc)[0]
fly_time_eq

T_{fly} = 2⋅hour

Number of arms

In [6]:
n_arms = symbols('n_{arms}')
n_arms_eq = Eq(n_arms,6)
n_arms_eq

n_{arms} = 6

Diameter of the drone

In [7]:
leng = symbols('D_{drone}')
leng_eq = Eq(leng,1800*unit.mm)
add_equation(leng_eq,leng)
leng_eq

D_{drone} = 1800⋅millimeter

-------------------------------------------------------------

## Main equations

### Total mass

In [8]:
m_propulsion = symbols('M_{propulsion}')
m_energy = symbols('M_{energy}')
m_structure = symbols('M_{structure}')
m_total = symbols('M_{total}')
m_total_eq = Eq(m_total,m_propulsion+m_energy+m_structure+m_payload)
add_equation(m_total_eq,m_total)
m_total_eq

M_{total} = M_{energy} + M_{payload} + M_{propulsion} + M_{structure}

### Total power consumption

In [9]:
p_propulsion = symbols('P_{propulsion}')
p_total = symbols('P_{total}')
p_total_eq = Eq(p_total,p_propulsion)
add_equation(p_total_eq,p_total)
p_total_eq

P_{total} = P_{propulsion}

-----------------------------------

### Thrust force

In [10]:
f_thrust_full = symbols('F_{thrust}^{full}')
f_thrust_full_eq_gee = Eq(f_thrust_full,2*m_total/unit.kg*9.81*unit.N)
f_thrust_full_eq_simple = convert_to(f_thrust_full_eq_gee,[unit.m,unit.s])
add_equation(f_thrust_full_eq_gee,f_thrust_full)
# sub_dict[f_thrust_full]=solve(f_thrust_full_calc)[0]
f_thrust_full_eq_gee

                     19.62⋅M_{total}⋅newton
F_{thrust}__{full} = ──────────────────────
                            kilogram       

-----------------------------------------

### Propulsion system

Mass of the system

In [11]:
m_propulsion_eq = Eq(m_propulsion,p_propulsion/(6000*unit.W/unit.kg))
add_equation(m_propulsion_eq,m_propulsion)
# sub_dict[m_propulsion]=solve(m_propulsion_calc)[0]
m_propulsion_eq

                 P_{propulsion}⋅kilogram
M_{propulsion} = ───────────────────────
                        6000⋅watt       

Power consumption

In [12]:
p_propulsion_eq = Eq(p_propulsion,f_thrust_full/(0.085*unit.N/unit.W))
# sub_dict[p_propulsion]=solve(p_propulsion_calc)[0]
add_equation(p_propulsion_eq,p_propulsion)
p_propulsion_eq

                 11.7647058823529⋅F_{thrust}__{full}⋅watt
P_{propulsion} = ────────────────────────────────────────
                                  newton                 

--------------------------------

### Energy system

In [32]:
k_stack_mass = symbols('k_{stack}')
k_stack_mass_eq = Eq(k_stack_mass,800*unit.W/unit.kg)
add_equation(k_stack_mass_eq,k_stack_mass)
k_stack_mass_eq

            800⋅watt
k_{stack} = ────────
            kilogram

In [33]:
m_stack = symbols('M_{stack}')
m_stack_eq = Eq(m_stack,p_total/k_stack_mass)
add_equation(m_stack_eq,m_stack)
m_stack_eq

            P_{total}
M_{stack} = ─────────
            k_{stack}

In [24]:
m_fuel = symbols('M_{fuel}')
m_fuel_eq = Eq(m_fuel,p_total)
m_fuel

M_{fuel}

In [31]:
m_energy_eq = Eq(m_energy,1.2*(m_stack+m_fuel))
m_energy_eq

M_{energy} = 1.2⋅M_{fuel} + 1.2⋅M_{stack}

In [13]:
m_energy_eq = Eq(m_energy,(p_total/unit.W/800+p_total*3600*fly_time_var/unit.W/141.9/10e6)*unit.kg)
add_equation(m_energy_eq,m_energy)
# sub_dict[m_energy]=solve(m_energy_calc)[0]
m_energy_eq

             0.00125507399577167⋅P_{total}⋅kilogram
M_{energy} = ──────────────────────────────────────
                              watt                 

--------------------------------------------

### Structure

#### Frame mass dependency

In [14]:
#1 ---- We can estimate mass of structure by experimental data
#Diameter 
x = np.array([1550, 1060, 605, 1840])
#Take-off weight
y = np.array([19, 8.25, 4, 40])
#Frame mass calculated by propulsion team 6*arm_weight+0.4kg for the middle part
z = np.array([1.24, 0.658, 0.472, 2.584])
A = np.vstack([x, y, np.ones_like(x)]).T
Q, R = np.linalg.qr(A)
a, b, c = np.linalg.inv(R)@Q.T@z
m_structure_eq = Eq(m_structure,(a*leng/unit.mm+b*m_total/unit.kg+c)*unit.kg)
add_equation(m_structure_eq,m_structure)
#sub_dict[m_structure]=solve(m_structure_calc)[0]
m_structure_eq
#2 ---- Or by a formula
#ro = 1600
#sigma_max = 300e6
#D_relative = 0.0018
#m_structure_eq = Eq(m_structure,(a*leng/unit.mm+b*m_total/unit.kg+c)*unit.kg)
#add_equation(m_structure_eq,m_structure)
#m_structure_eq

                ⎛                    0.000248391401913562⋅D_{drone}   0.067014
M_{structure} = ⎜0.358716261164815 - ────────────────────────────── + ────────
                ⎝                              millimeter                     

0840984393⋅M_{total}⎞         
────────────────────⎟⋅kilogram
  kilogram          ⎠         

----------------------------------

## Calculation results

In [15]:
eq_list = list(map(lambda c:c['eq'],d.values()))
var_list = list(map(lambda c:c['var'],d.values()))
temp_results = linsolve(eq_list,var_list)
results_list = []
for a in temp_results:
    for b in a:
        results_list.append(b)
    break
# results_list

In [16]:
save_results(results_list)

In [17]:
[m_total,d[m_total]['result']]

[M_{total}, 29.6150394427936⋅kilogram]

In [18]:
[m_propulsion,d[m_propulsion]['result']]

[M_{propulsion}, 1.13930798797571⋅kilogram]

In [19]:
[m_structure,d[m_structure]['result']]

[M_{structure}, 1.89623648151837⋅kilogram]

In [20]:
[m_energy,d[m_energy]['result']]

[M_{energy}, 8.57949497329951⋅kilogram]

In [21]:
d

{M_{payload}: {'eq': Eq(M_{payload}, 18*kilogram),
  'var': M_{payload},
  'result': 18*kilogram},
 T_{fly}: {'eq': Eq(T_{fly}, 2*hour), 'var': T_{fly}, 'result': 2*hour},
 D_{drone}: {'eq': Eq(D_{drone}, 1800*millimeter),
  'var': D_{drone},
  'result': 1800*millimeter},
 M_{total}: {'eq': Eq(M_{total}, M_{energy} + M_{payload} + M_{propulsion} + M_{structure}),
  'var': M_{total},
  'result': 29.6150394427936*kilogram},
 P_{total}: {'eq': Eq(P_{total}, P_{propulsion}),
  'var': P_{total},
  'result': 6835.84792785424*watt},
 F_{thrust}^{full}: {'eq': Eq(F_{thrust}^{full}, 19.62*newton*M_{total}/kilogram),
  'var': F_{thrust}^{full},
  'result': 581.04707386761*newton},
 M_{propulsion}: {'eq': Eq(M_{propulsion}, kilogram*P_{propulsion}/(6000*watt)),
  'var': M_{propulsion},
  'result': 1.13930798797571*kilogram},
 P_{propulsion}: {'eq': Eq(P_{propulsion}, 11.7647058823529*watt*F_{thrust}^{full}/newton),
  'var': P_{propulsion},
  'result': 6835.84792785424*watt},
 M_{energy}: {'eq': E